In [3]:
import pandas as pd
import os, sys, json
from tqdm import tqdm
root_dir = os.path.dirname(os.getcwd())
sys.path.append(root_dir)

# Datasets & Baselines

In [4]:

metadataset_df = pd.read_csv(f"{root_dir}/TabZilla/tutorials/metadataset_clean_andreas.csv")

def read_json(file):
    with open(file, "r") as f:
        return json.load(f)
    
eval = {}

model_dict = {
    'tabpfn': 'TabPFNModel',
    'tabflex': 'TabFlexModel',
    'tabfast': 'TabFastModel',
}

baselines = [
    'LinearModel', 
    'KNN', 
    'SVM', 
    'DecisionTree', 
    'RandomForest', 
    'XGBoost', 
    'CatBoost', 
    'LightGBM', 
    'MLP', 
    'TabNet', 
    'VIME', 
    'TabTransformer', 
    'NODE', 
    'DeepGBM', 
    'STG', 
    'NAM', 
    'DeepFM', 
    'SAINT', 
    'DANet', 
    'rtdl_MLP', 
    'rtdl_ResNet', 
    'rtdl_FTTransformer',
]

In [5]:
def get_datasets(table_idx):
    if table_idx == 1:
        from analysis.table1 import DATASETS
    elif table_idx == 2:
        from analysis.table2 import DATASETS
    elif table_idx == 4:
        from analysis.table4 import DATASETS
        
    return DATASETS

In [6]:
def get_dataset_stat(dataset):
    result_dir = f"{root_dir}/results/TabFlexModel/{dataset}"
    items = os.listdir(result_dir)
    for item in items:
        if item == 'default_trial0_results.json':
            result = read_json(f"{result_dir}/{item}")
            break
        elif os.path.isdir(f"{result_dir}/{item}"):
            result = read_json(f"{result_dir}/{item}/default_trial0_results.json")
            break
    return {
        'num_classes': result['dataset']['num_classes'],
        'num_features': result['dataset']['num_features'],
        'num_instances': result['dataset']['num_instances'],
    }

In [7]:
def get_results(dataset, model, metric = 'Accuracy'):
    result_dir = f"{root_dir}/results/{model_dict[model]}/{dataset}"
    items = os.listdir(result_dir)
    
    best_eval = None
    for item in items: 
        try:
            if item == 'default_trial0_results.json':
                eval = read_json(f"{result_dir}/{item}")
                if best_eval is None or eval['scorers']['test'][metric] > best_eval['scorers']['test'][metric]:
                    best_eval = eval
            elif os.path.isdir(f"{result_dir}/{item}"):
                eval = read_json(f"{result_dir}/{item}/default_trial0_results.json")
                if best_eval is None or eval['scorers']['test'][metric] > best_eval['scorers']['test'][metric]:
                    best_eval = eval
        except KeyboardInterrupt:
            raise KeyboardInterrupt
        except Exception as e:
            # print the error message
            print(e)
            print(f"Error reading {result_dir}/{item}")
                  
    return best_eval

In [8]:
def get_tabmodels(method, eval, raw_results, metric = 'Accuracy'):
    eval[method] = {}
    for dataset in raw_results[method]:
        eval[method][dataset] = {}
        metrics = raw_results[method][dataset]['scorers']['test'][metric]
        metrics = pd.Series(metrics)
        eval[method][dataset][f'median_{metric}'] = metrics.median()
        eval[method][dataset][f'mean_{metric}'] = metrics.mean()
        eval[method][dataset]['train_time'] = pd.Series(raw_results[method][dataset]['timers']['train'][1:]).mean()
        eval[method][dataset]['test_time'] = pd.Series(raw_results[method][dataset]['timers']['test']).mean()
        eval[method][dataset]['total_time'] = eval[method][dataset]['train_time'] + eval[method][dataset]['test_time']
    

In [9]:
def get_baselines(method, eval, DATASETS, metric = 'Accuracy'):
    eval[method] = {}
    for dataset in DATASETS:
        eval[method][dataset] = {}
        result = metadataset_df.loc[
            (metadataset_df["alg_name"] == method) & 
            (metadataset_df["hparam_source"] == "default") &
            (metadataset_df["dataset_name"] == dataset),
            [
                "dataset_fold_id", 
                "alg_name", 
                "hparam_source", 
                f"{metric}__test", 
                "training_time",
                'eval-time__test',]
        ]
        if result.empty:
            eval[method][dataset][f'median_{metric}'] = 0
            eval[method][dataset][f'mean_{metric}'] = 0
            eval[method][dataset]['train_time'] = 0
            eval[method][dataset]['test_time'] = 0
            eval[method][dataset]['total_time'] = 0
        else:
            eval[method][dataset][f'median_{metric}'] = result[f"{metric}__test"].median()
            eval[method][dataset][f'mean_{metric}'] = result[f"{metric}__test"].mean()
            eval[method][dataset]['train_time'] = result['training_time'].mean()
            eval[method][dataset]['test_time'] = result['eval-time__test'].mean()
            eval[method][dataset]['total_time'] = eval[method][dataset]['train_time'] + eval[method][dataset]['test_time']

# Get the Results of TabModels

In [10]:
table_idx = 1
metric = 'AUC'

DATASETS = get_datasets(table_idx)
raw_results = {}

tqdm_bar = tqdm(model_dict)
for model in tqdm_bar:
    tqdm_bar.set_description(f"Reading results of {model}...")
    raw_results[model] = {}
    for dataset in DATASETS:
        raw_results[model][dataset] = get_results(dataset, model, metric = metric)            
        
tqdm_bar = tqdm(model_dict)
for model in tqdm_bar:
    tqdm_bar.set_description(f"Computing results of {model}...")
    get_tabmodels(model, eval, raw_results, metric)

Computing results of tabfast...: 100%|██████████| 3/3 [00:00<00:00, 17.05it/s]


In [11]:
table_dict = {}

for dataset in DATASETS:
    table_dict[dataset] = get_dataset_stat(dataset)
    for method in model_dict:
        table_dict[dataset][f'{method}_{metric}'] = round(eval[method][dataset][f'mean_{metric}'], 4)
        table_dict[dataset][f'{method}_time'] = round(eval[method][dataset]['total_time'], 4)
        
table = pd.DataFrame(table_dict).T

table

,num_classes,num_features,num_instances,tabpfn_AUC,tabpfn_time,tabflex_AUC,tabflex_time,tabfast_AUC,tabfast_time
openml__visualizing_environmental__3602,1.0,3.0,111.0,0.6994,0.0231,0.7322,0.0746,0.6939,0.0288
openml__labor__4,1.0,16.0,57.0,0.9875,0.0652,0.9875,0.0635,1.0000,0.0629
openml__monks-problems-2__146065,1.0,6.0,601.0,1.0000,0.0903,0.7726,0.0402,0.9596,0.0402
openml__tic-tac-toe__49,1.0,9.0,958.0,0.9951,0.1971,0.8491,0.1877,0.9044,0.0582
openml__dermatology__35,6.0,34.0,366.0,0.9996,0.1648,0.9976,0.1291,0.9991,0.1266
...,...,...,...,...,...,...,...,...,...
openml__eucalyptus__2079,5.0,19.0,736.0,0.9298,0.1783,0.9086,0.0871,0.9275,0.0845
openml__libras__360948,10.0,104.0,360.0,0.5306,0.3544,0.7460,0.3281,0.5364,0.3145
openml__yeast__145793,4.0,8.0,1269.0,0.8781,0.2260,0.8729,0.1282,0.8719,0.0668
openml__cmc__23,3.0,9.0,1473.0,0.7366,0.2727,0.7329,0.0714,0.7288,0.0734


In [41]:
data_table = table[['num_features', 'num_instances', 'num_classes']].T
# for each column, only take the string between two "__"
data_table.columns = [col.split('__')[1] for col in data_table.columns]
data_table.T.sort_values(by='num_instances', ascending=False)

,num_features,num_instances,num_classes
poker-hand,10.0,1025009.0,10.0
airlines,7.0,539383.0,1.0
albert,78.0,425240.0,1.0
MiniBooNE,50.0,130064.0,1.0
higgs,28.0,98050.0,1.0
electricity,8.0,45312.0,1.0
jungle_chess_2pcs_raw_endgame_complete,6.0,44819.0,3.0
nomao,118.0,34465.0,1.0
guillermo,4296.0,20000.0,1.0
elevators,18.0,16599.0,1.0


In [42]:
table[(table[f'tabpfn_{metric}'] >= table[f'tabflex_{metric}']) & (table[f'tabpfn_{metric}'] >= table[f'tabfast_{metric}'])]

,num_classes,num_features,num_instances,tabpfn_AUC,tabpfn_time,tabflex_AUC,tabflex_time,tabfast_AUC,tabfast_time
openml__Australian__146818,1.0,14.0,690.0,0.9318,0.1900,0.9234,0.0709,0.9298,0.0692
openml__GesturePhaseSegmentationProcessed__14969,5.0,32.0,9873.0,0.8392,1.1072,0.7759,0.5931,0.8126,0.7722
openml__MiniBooNE__168335,1.0,50.0,130064.0,0.9769,3.1937,0.9659,6.2118,0.9740,7.2221
openml__artificial-characters__14964,10.0,7.0,10218.0,0.9601,0.9679,0.8991,0.2873,0.9500,0.2842
openml__balance-scale__11,3.0,4.0,625.0,0.9993,0.0999,0.9952,0.0350,0.9974,0.0343
openml__credit-approval__29,1.0,15.0,690.0,0.9353,0.1621,0.9326,0.0767,0.9333,0.0748
openml__credit-g__31,1.0,20.0,1000.0,0.7651,0.2235,0.7329,0.0981,0.7616,0.2307
openml__electricity__219,1.0,8.0,45312.0,0.8892,1.6534,0.8711,1.2358,0.8872,2.2287
openml__heart-h__50,1.0,13.0,294.0,0.8819,0.0837,0.8721,0.0566,0.8774,0.0554
openml__jungle_chess_2pcs_raw_endgame_complete__167119,3.0,6.0,44819.0,0.9466,1.6508,0.8958,1.1016,0.9269,1.1877


In [43]:
table[(table[f'tabflex_{metric}'] >= table[f'tabpfn_{metric}']) & (table[f'tabflex_{metric}'] >= table[f'tabfast_{metric}'])]

,num_classes,num_features,num_instances,tabpfn_AUC,tabpfn_time,tabflex_AUC,tabflex_time,tabfast_AUC,tabfast_time
openml__SpeedDating__146607,1.0,120.0,8378.0,0.5469,1.5849,0.8481,1.5263,0.5608,1.2764
openml__audiology__7,24.0,69.0,226.0,0.8182,0.2328,0.8452,0.2177,0.8222,0.8767
openml__cnae-9__9981,9.0,856.0,1080.0,0.4794,0.5066,0.9136,3.0985,0.5028,0.9265
openml__colic__25,1.0,26.0,368.0,0.8784,0.1427,0.8802,0.1005,0.8751,0.1030
openml__guillermo__168337,1.0,4296.0,20000.0,0.4999,1.7556,0.5017,22.3321,0.4991,3.1121
openml__jasmine__168911,1.0,144.0,2984.0,0.7040,1.2732,0.8511,0.7668,0.7620,0.4234
openml__nomao__9977,1.0,118.0,34465.0,0.7623,1.8179,0.9864,4.1674,0.7568,2.7171
openml__one-hundred-plants-texture__9956,100.0,64.0,1599.0,0.5452,0.5155,0.5483,0.3124,0.5453,0.3066


In [44]:
table[(table[f'tabfast_{metric}'] >= table[f'tabpfn_{metric}']) & (table[f'tabfast_{metric}'] >= table[f'tabflex_{metric}'])]

,num_classes,num_features,num_instances,tabpfn_AUC,tabpfn_time,tabflex_AUC,tabflex_time,tabfast_AUC,tabfast_time
openml__Bioresponse__9910,1.0,1776.0,3751.0,0.4980,1.2862,0.4952,5.3101,0.5450,0.7605
openml__ada_agnostic__3896,1.0,48.0,4562.0,0.8863,1.0700,0.8863,0.3697,0.8958,0.3492
openml__airlines__189354,1.0,7.0,539383.0,0.6279,9.7280,0.6214,3.7808,0.6393,4.1948
openml__albert__189356,1.0,78.0,425240.0,0.6860,9.3899,0.6914,13.1747,0.7011,13.4606
openml__elevators__3711,1.0,18.0,16599.0,0.9470,1.7780,0.9428,0.6259,0.9492,0.5989
openml__higgs__146606,1.0,28.0,98050.0,0.7241,2.8184,0.6814,5.0943,0.7560,4.9200
openml__poker-hand__9890,10.0,10.0,1025009.0,0.7176,15.3640,0.5559,4.5662,0.8372,4.8793
openml__profb__3561,1.0,9.0,672.0,0.6617,0.2318,0.6460,0.0580,0.6644,0.0527


In [45]:
table_fast = table[(table['num_instances'] >= 50000) & (table['num_features'] <= 100)].drop(columns = ['tabflex_time', f'tabflex_{metric}'])
table_fast

,num_classes,num_features,num_instances,tabpfn_AUC,tabpfn_time,tabfast_AUC,tabfast_time
openml__MiniBooNE__168335,1.0,50.0,130064.0,0.9769,3.1937,0.9740,7.2221
openml__airlines__189354,1.0,7.0,539383.0,0.6279,9.7280,0.6393,4.1948
openml__albert__189356,1.0,78.0,425240.0,0.6860,9.3899,0.7011,13.4606
openml__higgs__146606,1.0,28.0,98050.0,0.7241,2.8184,0.7560,4.9200
openml__poker-hand__9890,10.0,10.0,1025009.0,0.7176,15.3640,0.8372,4.8793


In [46]:
table_flex = table[(table['num_features'] > 100) | (table['num_features']/table['num_instances'] >= 0.2)].drop(columns = ['tabfast_time', f'tabfast_{metric}'])
table_flex

,num_classes,num_features,num_instances,tabpfn_AUC,tabpfn_time,tabflex_AUC,tabflex_time
openml__Bioresponse__9910,1.0,1776.0,3751.0,0.4980,1.2862,0.4952,5.3101
openml__SpeedDating__146607,1.0,120.0,8378.0,0.5469,1.5849,0.8481,1.5263
openml__audiology__7,24.0,69.0,226.0,0.8182,0.2328,0.8452,0.2177
openml__cnae-9__9981,9.0,856.0,1080.0,0.4794,0.5066,0.9136,3.0985
openml__guillermo__168337,1.0,4296.0,20000.0,0.4999,1.7556,0.5017,22.3321
openml__jasmine__168911,1.0,144.0,2984.0,0.7040,1.2732,0.8511,0.7668
openml__nomao__9977,1.0,118.0,34465.0,0.7623,1.8179,0.9864,4.1674


# Compare with Other Baselines

In [15]:
tqdm_bar = tqdm(baselines)
for baseline in tqdm_bar:
    tqdm_bar.set_description(f"Computing results of {baseline}...") 
    try:
        get_baselines(baseline, eval, DATASETS, metric)
    except:
        print(f"Error in {baseline}")

methods = baselines + list(model_dict.keys())
final_results = {}

for method in methods:
    result = pd.DataFrame(eval[method]).T
    final_results[method] = result[f'mean_{metric}'].mean()
    
sorted_methods = sorted(methods, key=lambda x: final_results[x], reverse=True)
for method in sorted_methods:
    print(f"| {method} | {final_results[method]} |")

Computing results of rtdl_FTTransformer...: 100%|██████████| 22/22 [02:07<00:00,  5.78s/it]

| XGBoost | 0.89626674151231 |
| CatBoost | 0.8710780319364649 |
| rtdl_ResNet | 0.865423274822542 |
| RandomForest | 0.8517725838051355 |
| rtdl_MLP | 0.8389464580267786 |
| tabflex | 0.828352668987128 |
| tabfast | 0.8265780311619968 |
| tabpfn | 0.8214072715506934 |
| MLP | 0.8144863457854493 |
| LightGBM | 0.8064431201510455 |
| DecisionTree | 0.7904649113211598 |
| LinearModel | 0.7844321715770098 |
| KNN | 0.7474054796492654 |
| NODE | 0.7247265454837093 |
| TabNet | 0.7219269236127556 |
| VIME | 0.688526351681953 |
| rtdl_FTTransformer | 0.6791009939981121 |
| STG | 0.6748985179731256 |
| DANet | 0.660580705297897 |
| SVM | 0.6507085992457963 |
| TabTransformer | 0.5891746586557063 |
| SAINT | 0.5367639924497778 |
| DeepFM | 0.5026993451840268 |
| NAM | 0.32484285783465794 |
| DeepGBM | 0.0 |


In [47]:
for dataset in DATASETS:
    table_dict[dataset] = get_dataset_stat(dataset)
    for method in baselines + list(model_dict.keys()):
        table_dict[dataset][f'{method}_{metric}'] = round(eval[method][dataset][f'mean_{metric}'], 4)
        table_dict[dataset][f'{method}_time'] = round(eval[method][dataset]['total_time'], 4)

In [48]:
full_table = pd.DataFrame(table_dict).T

In [49]:
full_table[full_table['num_features'] >= 1000]

,num_classes,num_features,num_instances,LinearModel_AUC,LinearModel_time,KNN_AUC,KNN_time,SVM_AUC,SVM_time,DecisionTree_AUC,DecisionTree_time,RandomForest_AUC,RandomForest_time,XGBoost_AUC,XGBoost_time,CatBoost_AUC,CatBoost_time,LightGBM_AUC,LightGBM_time,MLP_AUC,MLP_time,TabNet_AUC,TabNet_time,VIME_AUC,VIME_time,TabTransformer_AUC,TabTransformer_time,NODE_AUC,NODE_time,DeepGBM_AUC,DeepGBM_time,STG_AUC,STG_time,NAM_AUC,NAM_time,DeepFM_AUC,DeepFM_time,SAINT_AUC,SAINT_time,DANet_AUC,DANet_time,rtdl_MLP_AUC,rtdl_MLP_time,rtdl_ResNet_AUC,rtdl_ResNet_time,rtdl_FTTransformer_AUC,rtdl_FTTransformer_time,tabpfn_AUC,tabpfn_time,tabflex_AUC,tabflex_time,tabfast_AUC,tabfast_time
openml__Bioresponse__9910,1.0,1776.0,3751.0,0.8093,0.2508,0.8059,12.7006,0.8521,208.5077,0.7797,0.8658,0.8194,0.8952,0.8700,2.4921,0.8555,2.6371,0.8765,19.7552,0.8293,6.1119,0.8145,57.4285,0.8184,419.4598,0.8464,8.0684,0.8320,89.0104,0.0,0.0,0.8350,39.8958,0.0,0.0,0.8128,14.9921,0.0,0.0,0.8398,89.7274,0.8323,4.7665,0.8335,5.8221,0.0,0.0,0.4980,1.2862,0.4952,5.3101,0.5450,0.7605
openml__guillermo__168337,1.0,4296.0,20000.0,0.7069,3.6900,0.0000,0.0000,0.0000,0.0000,0.7455,47.7885,0.8195,22.3861,0.8874,13.1522,0.0000,0.0000,0.0000,0.0000,0.7577,33.6682,0.7076,340.3815,0.0000,0.0000,0.7391,81.7504,0.6523,248.2201,0.0,0.0,0.4708,212.8284,0.0,0.0,0.7285,239.0068,0.0,0.0,0.0000,0.0000,0.7481,36.8471,0.7762,43.4568,0.0,0.0,0.4999,1.7556,0.5017,22.3321,0.4991,3.1121


In [60]:
# show full columns
pd.set_option('display.max_columns', None)
this_table = full_table.loc[table_flex.index].drop(columns = ['tabfast_time', f'tabfast_{metric}'])
# ['tabpfn', 'tabflex', 'LinearModel', 'DecisionTree', 'XGBoost', 'MLP', 'TabNet', 'NODE', 'STG', 'rtdl_ResNet']
selected_methods = ['CatBoost', 'MLP', 'TabNet', 'rtdl_ResNet', 'tabpfn', 'tabflex', ]
# selected_methods = ['tabpfn', 'tabflex', 'NODE', 'STG', 'rtdl_ResNet']
columns = ['num_classes', 'num_features', 'num_instances'] + [f'{method}_{metric}' for method in selected_methods] + [f'{method}_time' for method in selected_methods]
this_table = this_table[columns]
this_table


,num_classes,num_features,num_instances,CatBoost_AUC,MLP_AUC,TabNet_AUC,rtdl_ResNet_AUC,tabpfn_AUC,tabflex_AUC,CatBoost_time,MLP_time,TabNet_time,rtdl_ResNet_time,tabpfn_time,tabflex_time
openml__Bioresponse__9910,1.0,1776.0,3751.0,0.8555,0.8293,0.8145,0.8335,0.4980,0.4952,2.6371,6.1119,57.4285,5.8221,1.2862,5.3101
openml__SpeedDating__146607,1.0,120.0,8378.0,0.8637,0.8460,0.8286,0.8626,0.5469,0.8481,12.0404,13.0573,150.6250,11.0074,1.5849,1.5263
openml__audiology__7,24.0,69.0,226.0,0.8752,0.6498,0.6157,0.9229,0.8182,0.8452,54.5559,5.1219,7.4409,4.9215,0.2328,0.2177
openml__cnae-9__9981,9.0,856.0,1080.0,0.9899,0.9964,0.9841,0.9944,0.4794,0.9136,1.7671,10.5023,26.5780,5.5932,0.5066,3.0985
openml__guillermo__168337,1.0,4296.0,20000.0,0.0000,0.7577,0.7076,0.7762,0.4999,0.5017,0.0000,33.6682,340.3815,43.4568,1.7556,22.3321
openml__jasmine__168911,1.0,144.0,2984.0,0.8712,0.8516,0.8226,0.8538,0.7040,0.8511,1.6779,6.0579,61.8220,4.4049,1.2732,0.7668
openml__nomao__9977,1.0,118.0,34465.0,0.9915,0.9913,0.9923,0.9929,0.7623,0.9864,4.2106,86.4505,517.7118,70.0467,1.8179,4.1674


In [61]:
this_table = this_table.astype({'num_classes': int, 'num_features': int, 'num_instances': int})
# for the index, only keep the str between two "__"s
this_table.index = this_table.index.map(lambda x: x.split('__')[1])
# for the columns, replace the "_" with " "
this_table.columns = this_table.columns.str.replace('_', ' ')
this_table.columns = this_table.columns.str.replace('rtdl ', '')
# for floats, round to 3 decimal places when convert the talbe to latex
# make all columns align to the center
print(this_table.to_latex(float_format="%.3f", column_format='c'*len(this_table.columns), escape=False))

\begin{tabular}{ccccccccccccccc}
\toprule
 & num classes & num features & num instances & CatBoost AUC & MLP AUC & TabNet AUC & ResNet AUC & tabpfn AUC & tabflex AUC & CatBoost time & MLP time & TabNet time & ResNet time & tabpfn time & tabflex time \\
\midrule
Bioresponse & 1 & 1776 & 3751 & 0.856 & 0.829 & 0.815 & 0.834 & 0.498 & 0.495 & 2.637 & 6.112 & 57.428 & 5.822 & 1.286 & 5.310 \\
SpeedDating & 1 & 120 & 8378 & 0.864 & 0.846 & 0.829 & 0.863 & 0.547 & 0.848 & 12.040 & 13.057 & 150.625 & 11.007 & 1.585 & 1.526 \\
audiology & 24 & 69 & 226 & 0.875 & 0.650 & 0.616 & 0.923 & 0.818 & 0.845 & 54.556 & 5.122 & 7.441 & 4.921 & 0.233 & 0.218 \\
cnae-9 & 9 & 856 & 1080 & 0.990 & 0.996 & 0.984 & 0.994 & 0.479 & 0.914 & 1.767 & 10.502 & 26.578 & 5.593 & 0.507 & 3.099 \\
guillermo & 1 & 4296 & 20000 & 0.000 & 0.758 & 0.708 & 0.776 & 0.500 & 0.502 & 0.000 & 33.668 & 340.382 & 43.457 & 1.756 & 22.332 \\
jasmine & 1 & 144 & 2984 & 0.871 & 0.852 & 0.823 & 0.854 & 0.704 & 0.851 & 1.678 & 6.058 & 

In [64]:
this_table = full_table.loc[table_fast.index].drop(columns = ['tabflex_time', f'tabflex_{metric}'])
# ['tabpfn', 'tabflex', 'LinearModel', 'DecisionTree', 'XGBoost', 'MLP', 'TabNet', 'NODE', 'STG', 'rtdl_ResNet']
selected_methods = ['CatBoost', 'MLP', 'TabNet', 'rtdl_ResNet', 'tabpfn', 'tabfast', ]
columns = ['num_classes', 'num_features', 'num_instances'] + [f'{method}_{metric}' for method in selected_methods] + [f'{method}_time' for method in selected_methods]
this_table = this_table[columns]
this_table

,num_classes,num_features,num_instances,CatBoost_AUC,MLP_AUC,TabNet_AUC,rtdl_ResNet_AUC,tabpfn_AUC,tabfast_AUC,CatBoost_time,MLP_time,TabNet_time,rtdl_ResNet_time,tabpfn_time,tabfast_time
openml__MiniBooNE__168335,1.0,50.0,130064.0,0.9778,0.9782,0.927,0.9692,0.9769,0.9740,2.6046,265.5586,699.0828,447.4033,3.1937,7.2221
openml__airlines__189354,1.0,7.0,539383.0,0.6977,0.6509,0.000,0.7047,0.6279,0.6393,4.5096,511.9868,0.0000,571.4248,9.7280,4.1948
openml__albert__189356,1.0,78.0,425240.0,0.7677,0.6542,0.000,0.7562,0.6860,0.7011,33.9801,405.5280,0.0000,490.1059,9.3899,13.4606
openml__higgs__146606,1.0,28.0,98050.0,0.7871,0.7931,0.000,0.8119,0.7241,0.7560,1.9992,222.4406,0.0000,199.2417,2.8184,4.9200
openml__poker-hand__9890,10.0,10.0,1025009.0,0.8385,0.5393,0.000,0.0000,0.7176,0.8372,355.3651,504.5200,0.0000,0.0000,15.3640,4.8793


In [65]:
this_table = this_table.astype({'num_classes': int, 'num_features': int, 'num_instances': int})
# for the index, only keep the str between two "__"s
this_table.index = this_table.index.map(lambda x: x.split('__')[1])
# for the columns, replace the "_" with " "
this_table.columns = this_table.columns.str.replace('_', ' ')
this_table.columns = this_table.columns.str.replace('rtdl ', '')
# for floats, round to 3 decimal places when convert the talbe to latex
# make all columns align to the center
print(this_table.to_latex(float_format="%.3f", column_format='c'*len(this_table.columns), escape=False))

\begin{tabular}{ccccccccccccccc}
\toprule
 & num classes & num features & num instances & CatBoost AUC & MLP AUC & TabNet AUC & ResNet AUC & tabpfn AUC & tabfast AUC & CatBoost time & MLP time & TabNet time & ResNet time & tabpfn time & tabfast time \\
\midrule
MiniBooNE & 1 & 50 & 130064 & 0.978 & 0.978 & 0.927 & 0.969 & 0.977 & 0.974 & 2.605 & 265.559 & 699.083 & 447.403 & 3.194 & 7.222 \\
airlines & 1 & 7 & 539383 & 0.698 & 0.651 & 0.000 & 0.705 & 0.628 & 0.639 & 4.510 & 511.987 & 0.000 & 571.425 & 9.728 & 4.195 \\
albert & 1 & 78 & 425240 & 0.768 & 0.654 & 0.000 & 0.756 & 0.686 & 0.701 & 33.980 & 405.528 & 0.000 & 490.106 & 9.390 & 13.461 \\
higgs & 1 & 28 & 98050 & 0.787 & 0.793 & 0.000 & 0.812 & 0.724 & 0.756 & 1.999 & 222.441 & 0.000 & 199.242 & 2.818 & 4.920 \\
poker-hand & 10 & 10 & 1025009 & 0.839 & 0.539 & 0.000 & 0.000 & 0.718 & 0.837 & 355.365 & 504.520 & 0.000 & 0.000 & 15.364 & 4.879 \\
\bottomrule
\end{tabular}



In [22]:
method = 'TabPFNModel'
dataset = 'openml__lymph__10'

for time_type in ['training_time', 'eval-time__test']:
    print(time_type, metadataset_df.loc[
        (metadataset_df["alg_name"] == method) & 
        (metadataset_df["hparam_source"] == "default") &
        (metadataset_df["dataset_name"] == dataset),
        [
            "dataset_fold_id", 
            "alg_name", 
            "hparam_source", 
            f"{metric}__test", 
            "training_time",
            "eval-time__test"]
    ][time_type].mean())

training_time 0.00043345699999989006
eval-time__test 0.1568050029


In [ ]:
metadataset_df.loc[
    (metadataset_df["alg_name"] == method) & 
    (metadataset_df["hparam_source"] == "default") &
    (metadataset_df["dataset_name"] == dataset),
    [
        "dataset_fold_id", 
        "alg_name", 
        "hparam_source", 
        f"{metric}__test", 
        "training_time",
        "eval-time__test"]
]["eval-time__test"]

66390    0.031840
66646    0.014282
66902    0.022972
67158    0.030814
67414    0.030835
67670    0.021542
67926    0.032461
68182    0.022469
68438    0.029921
68694    0.023271
Name: eval-time__test, dtype: float64

In [ ]:
metadataset_df.loc[
    (metadataset_df["alg_name"] == method) & 
    (metadataset_df["hparam_source"] == "default") &
    (metadataset_df["dataset_name"] == dataset),
    [
        "dataset_fold_id", 
        "alg_name", 
        "hparam_source", 
        f"{metric}__test", 
        "training_time",
        "eval-time__test"]
]["training_time"]

66390    3.497594
66646    2.381082
66902    2.475568
67158    2.398105
67414    2.505245
67670    2.597106
67926    2.478758
68182    2.374573
68438    2.570568
68694    2.507408
Name: training_time, dtype: float64